In [1]:
# !pip install gradio
# !pip install streamlit
# !pip install openai==0.28

# !pip install gradio

# !pip install pandas._typing #may or may not be necessary

In [11]:
import sqlite3  # For SQLite
import psycopg2  # For PostgreSQL
import pymysql  # For MySQL
# import pyodbc  # For Microsoft SQL Server

import csv
import pandas as pd
import numpy as np

## Loading Sample Dataset

In [63]:
df = pd.read_csv('Amazon Sale Report.csv')
df.head()

/Users/angus/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,index,Order ID,Date,Status,Fulfilment,Sales Channel,ship-service-level,Style,SKU,Category,...,currency,Amount,ship-city,ship-state,ship-postal-code,ship-country,promotion-ids,B2B,fulfilled-by,Unnamed: 22
0,0,405-8078784-5731545,04-30-22,Cancelled,Merchant,Amazon.in,Standard,SET389,SET389-KR-NP-S,Set,...,INR,647.62,MUMBAI,MAHARASHTRA,400081.0,IN,NaN,False,Easy Ship,NaN
1,1,171-9198151-1101146,04-30-22,Shipped - Delivered to Buyer,Merchant,Amazon.in,Standard,JNE3781,JNE3781-KR-XXXL,kurta,...,INR,406.00,BENGALURU,KARNATAKA,560085.0,IN,Amazon PLCC Free-Financing Universal Merchant ...,False,Easy Ship,NaN
2,2,404-0687676-7273146,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3371,JNE3371-KR-XL,kurta,...,INR,329.00,NAVI MUMBAI,MAHARASHTRA,410210.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,True,NaN,NaN
3,3,403-9615377-8133951,04-30-22,Cancelled,Merchant,Amazon.in,Standard,J0341,J0341-DR-L,Western Dress,...,INR,753.33,PUDUCHERRY,PUDUCHERRY,605008.0,IN,NaN,False,Easy Ship,NaN
4,4,407-1069790-7240320,04-30-22,Shipped,Amazon,Amazon.in,Expedited,JNE3671,JNE3671-TU-XXXL,Top,...,INR,574.00,CHENNAI,TAMIL NADU,600073.0,IN,NaN,False,NaN,NaN


In [70]:
## Checking Types of Data in the dataset

df.dtypes

index                          int64
Order ID                      object
Date                  datetime64[ns]
Status                        object
Fulfilment                    object
Sales Channel                 object
ship-service-level            object
Style                         object
SKU                           object
Category                      object
Size                          object
ASIN                          object
Courier Status                object
Qty                            int64
currency                      object
Amount                       float64
ship-city                     object
ship-state                    object
ship-postal-code             float64
ship-country                  object
promotion-ids                 object
B2B                             bool
fulfilled-by                  object
Unnamed: 22                   object
dtype: object

In [72]:
## Converting Dates to Datetime objects

df["Date"] = pd.to_datetime(df['Date'])

In [76]:
## Checking range of date values in the dataset

df[["Date"]].describe()

<ipython-input-76-c52d5a49707e>:3: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df[["Date"]].describe()


,Date
count,128975
unique,91
top,2022-05-03 00:00:00
freq,2085
first,2022-03-31 00:00:00
last,2022-06-29 00:00:00


In [77]:
## Checking general range of numerical data values in the dataset

df.describe()

,index,Qty,Amount,ship-postal-code
count,128975.000000,128975.000000,121180.000000,128942.000000
mean,64487.000000,0.904431,648.561465,463966.236509
std,37232.019822,0.313354,281.211687,191476.764941
min,0.000000,0.000000,0.000000,110001.000000
25%,32243.500000,1.000000,449.000000,382421.000000
50%,64487.000000,1.000000,605.000000,500033.000000
75%,96730.500000,1.000000,788.000000,600024.000000
max,128974.000000,15.000000,5584.000000,989898.000000


In [78]:
## Types of currencies in the dataset

df['currency'].unique()

array(['INR', nan], dtype=object)

In [81]:
df['Style'].unique()

array(['SET389', 'JNE3781', 'JNE3371', ..., 'SET066', 'J0286', 'SET045'],
      dtype=object)

## Converting Dataset to Database

In [4]:
conn = sqlite3.connect('mydatabase.db')  # Connect to an SQLite database file or create one
cursor = conn.cursor() # need the cursor to be able to execute SQL commands in Python

In [5]:
def create_sqlite_db_from_csv(csv, database_name, table_name):
    conn = sqlite3.connect(database_name)
    with open(csv, "r") as file:
        data_df = pd.read_csv(file)

    data_df.to_sql(table_name, conn, if_exists="replace", index=False) ## ensure duplicate dataframe is not created
    conn.commit()
    conn.close() ## Always close connection after use 
    print(f"Database {database_name} with table {table_name} created successfully!")

create_sqlite_db_from_csv('Amazon Sale Report.csv', 'mydatabase.db', 'amzn')

<ipython-input-5-32a377473731>:13: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  create_sqlite_db_from_csv('Amazon Sale Report.csv', 'mydatabase.db', 'amzn')
/Users/angus/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:2869: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


Database mydatabase.db with table amzn created successfully!


## Connecting Database to Chatbot

In [82]:
import dotenv
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_sql_query_chain

from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain_experimental.sql import SQLDatabaseChain

import gradio as gr
import openai

In [85]:
# Creating API Key file #

# Uncomment cell and insert OpenAI API key in "your key here"

# f = open("creds.env", "w")
# f.write("your key here")
# f.close()

In [86]:
dotenv.load_dotenv("creds.env")

True

In [87]:
def chat(input):
    
    db = SQLDatabase.from_uri("sqlite:///mydatabase.db")
    
    llm = OpenAI(temperature=0, verbose=True)
    db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
    result = db_chain.run(input)
    return result

def chatbot(input, history=[]):
    
    db = SQLDatabase.from_uri("sqlite:///mydatabase.db")
    
    output = chat(input)
    
    chain = create_sql_query_chain(ChatOpenAI(temperature=0), db)
    response = chain.invoke({"question": input})
    f
    output += "\n\n Here is the SQL command used to complete your query: \n"
    
    output += response
    
    history.append((input, output))
    return output


gr.Interface(fn=chatbot,
             inputs="text",
             outputs="text").launch(debug=False)


Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.




> Entering new SQLDatabaseChain chain...
how much currency was exchanged in april?
SQLQuery:SELECT SUM("Amount") FROM "Amazon Sale Report" WHERE "Date" LIKE '%04-%';
SQLResult: [(30940548.58999985,)]
Answer:30940548.58999985
> Finished chain.


> Entering new SQLDatabaseChain chain...
how much currency was exchanged in april? convert the amount to US dollars
SQLQuery:SELECT currency, SUM(Amount) * (SELECT rate FROM currency_rates WHERE currency = 'INR' AND date = date('now')) AS "Amount in USD" FROM "Amazon Sale Report" WHERE Date LIKE '%04-%' GROUP BY currency LIMIT 5;

Traceback (most recent call last):
  File "/Users/angus/opt/anaconda3/lib/python3.8/site-packages/sqlalchemy/engine/base.py", line 1969, in _exec_single_context
    self.dialect.do_execute(
  File "/Users/angus/opt/anaconda3/lib/python3.8/site-packages/sqlalchemy/engine/default.py", line 922, in do_execute
    cursor.execute(statement, parameters)
sqlite3.OperationalError: no such table: currency_rates

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/angus/opt/anaconda3/lib/python3.8/site-packages/gradio/routes.py", line 534, in predict
    output = await route_utils.call_process_api(
  File "/Users/angus/opt/anaconda3/lib/python3.8/site-packages/gradio/route_utils.py", line 226, in call_process_api
    output = await app.get_blocks().process_api(
  File "/Users/angus/opt/anaconda3/lib/python3.8/site-packages/gradio/blocks.py", line 1550, in process_api
    result = await self.call_function(
  File "/Users/angus/op



> Entering new SQLDatabaseChain chain...
how much currency was exchanged in april? convert the amount from rupees to US dollars
SQLQuery:SELECT "currency", SUM("Amount") * (SELECT rate FROM currency_rates WHERE currency = 'INR' AND date = date('now')) AS "Amount in USD" FROM "Amazon Sale Report" WHERE "Date" LIKE '%04-%' GROUP BY "currency" LIMIT 5;

Traceback (most recent call last):
  File "/Users/angus/opt/anaconda3/lib/python3.8/site-packages/sqlalchemy/engine/base.py", line 1969, in _exec_single_context
    self.dialect.do_execute(
  File "/Users/angus/opt/anaconda3/lib/python3.8/site-packages/sqlalchemy/engine/default.py", line 922, in do_execute
    cursor.execute(statement, parameters)
sqlite3.OperationalError: no such table: currency_rates

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/angus/opt/anaconda3/lib/python3.8/site-packages/gradio/routes.py", line 534, in predict
    output = await route_utils.call_process_api(
  File "/Users/angus/opt/anaconda3/lib/python3.8/site-packages/gradio/route_utils.py", line 226, in call_process_api
    output = await app.get_blocks().process_api(
  File "/Users/angus/opt/anaconda3/lib/python3.8/site-packages/gradio/blocks.py", line 1550, in process_api
    result = await self.call_function(
  File "/Users/angus/op



> Entering new SQLDatabaseChain chain...
how much currency was exchanged in april? 
SQLQuery:SELECT SUM("Amount") FROM "Amazon Sale Report" WHERE "Date" LIKE '%04-%';
SQLResult: [(30940548.58999985,)]
Answer:30940548.58999985
> Finished chain.


> Entering new SQLDatabaseChain chain...
how much in indian rupees was exchanged in april? convert the amount  to US dollars
SQLQuery:SELECT SUM("Amount") * 0.013 FROM "Amazon Sale Report" WHERE "Date" LIKE '%04-%' AND "currency" = 'INR' LIMIT 5;
SQLResult: [(402227.13166999805,)]
Answer:The total amount exchanged in Indian Rupees in April is 402227.13 US Dollars.
> Finished chain.


> Entering new SQLDatabaseChain chain...
how much in indian rupees was exchanged in april? 
SQLQuery:SELECT SUM("Amount") FROM "Amazon Sale Report" WHERE "Date" LIKE '%04-%' AND "currency" = 'INR'
SQLResult: [(30940548.58999985,)]
Answer:30940548.58999985 Indian Rupees were exchanged in April.
> Finished chain.


> Entering new SQLDatabaseChain chain...
how much 